# Machine Learning Pipeline

## Dataset Download

In [2]:
import kaggle

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json /home/ubuntu/.kaggle/kaggle.json

mkdir: cannot create directory ‘/home/ubuntu/.kaggle’: File exists


In [5]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

  0%|                                                | 0.00/199k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 199k/199k [00:00<00:00, 50.1MB/s]


In [11]:
!unzip house-prices-advanced-regression-techniques.zip -d Datasets/

Archive:  house-prices-advanced-regression-techniques.zip
  inflating: Datasets/data_description.txt  
  inflating: Datasets/sample_submission.csv  
  inflating: Datasets/test.csv       
  inflating: Datasets/train.csv      


## Package Imports

In [161]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import seaborn as sns
pd.pandas.set_option('display.max_columns',None)

In [16]:
dataset = pd.read_csv('Datasets/train.csv')
print(dataset.shape)
dataset.head()

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Data Cleaning

In [181]:
def fill_nans (df,Label,drop_cols=[]):
    
    from scipy.stats import shapiro

    if drop_cols==[]:
        drop_cols=[Label]
    label_data = df[Label]
    # Replace the Nans of categorical variables
    cat_cols = list(df.drop(drop_cols,axis=1).select_dtypes(include=['object']))
    data = df.copy().drop(drop_cols,axis=1)
    data[cat_cols] = data[cat_cols].fillna('Missing')
    
    # Replace all cateogrical values that occuer less than 1% in a column
    for feature in cat_cols:
        temp = df.groupby(feature)[Label].count()/len(df)
        temp_df = temp[temp>0.01].index
        data[feature] = np.where(data[feature].isin(temp_df),data[feature],'Rare_or_Missing')
    
    # Replace the Nans of Numerical variables
    # Replace it with Mean (parametric) or Median (non-parametric) distribution of the feature.
    num_cols = list(df.drop(drop_cols,axis=1).select_dtypes(include=['int','double']))
    for feature in num_cols:
        
        # normality test
        stat, p = shapiro(data[feature])
        if p<=0.05:
            replace_value = data[feature].median()
        else:
            replace_value = data[feature].mean()
        
        # create new colum for future relevance checking
        data[feature+'_nan'] = np.where(dataset[feature].isnull(),1,0)
        data[feature].fillna(replace_value,inplace=True)
    
    data[Label] = label_data
    
    return {'data':data,'num_cols':num_cols,'cat_cols':cat_cols}

In [182]:
cleaning_results =  fill_nans(dataset,'SalePrice')
clean_data = cleaning_results['data']

In [158]:
clean_data.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Id_nan,MSSubClass_nan,LotFrontage_nan,LotArea_nan,OverallQual_nan,OverallCond_nan,YearBuilt_nan,YearRemodAdd_nan,MasVnrArea_nan,BsmtFinSF1_nan,BsmtFinSF2_nan,BsmtUnfSF_nan,TotalBsmtSF_nan,1stFlrSF_nan,2ndFlrSF_nan,LowQualFinSF_nan,GrLivArea_nan,BsmtFullBath_nan,BsmtHalfBath_nan,FullBath_nan,HalfBath_nan,BedroomAbvGr_nan,KitchenAbvGr_nan,TotRmsAbvGrd_nan,Fireplaces_nan,GarageYrBlt_nan,GarageCars_nan,GarageArea_nan,WoodDeckSF_nan,OpenPorchSF_nan,EnclosedPorch_nan,3SsnPorch_nan,ScreenPorch_nan,PoolArea_nan,MiscVal_nan,MoSold_nan,YrSold_nan,SalePrice
0,1,60,RL,65.000000,8450,Pave,Rare_or_Missing,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,Rare_or_Missing,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,Rare_or_Missing,Rare_or_Missing,Rare_or_Missing,0,2,2008,WD,Normal,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,208500
1,2,20,RL,80.000000,9600,Pave,Rare_or_Missing,Reg,Lvl,AllPub,FR2,Gtl,Rare_or_Missing,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,Rare_or_Missing,Rare_or_Missing,Rare_or_Missing,0,5,2007,WD,Normal,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,181500
2,3,60,RL,68.000000,11250,Pave,Rare_or_Missing,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,Rare_or_Missing,Rare_or_Missing,Rare_or_Missing,0,9,2008,WD,Normal,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,223500
3,4,70,RL,60.000000,9550,Pave,Rare_or_Missing,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,Rare_or_Missing,Rare_or_Missing,Rare_or_Missing,0,2,2006,WD,Abnorml,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,140000
4,5,60,RL,84.000000,14260,Pave,Rare_or_Missing,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,Rare_or_Missing,Rare_or_Missing,Rare_or_Missing,0,12,2008,WD,Normal,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,250000
5,6,50,RL,85.000000,14115,Pave,Rare_or_Missing,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,Rare_or_Missing,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,Ex,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,Rare_or_Missing,Attchd,1993.0,Unf,2,480,TA,TA,Y,40,30,

## Feature Engineering

In [265]:
def data_split(df,Label,val_need=False):
    
    from sklearn.model_selection import KFold, cross_val_score, train_test_split

    X = df.loc[:, df.columns != Label]
    Y = df[Label]
    
    # One small if loop for regression or classificaiton
    if Y.dtype is 'O':
        X_Train, X_Test, Y_Train, Y_Test = train_test_split(X,Y, test_size=0.3, random_state=42, shuffle=True, stratify=Y)
    else:
        X_Train, X_Test, Y_Train, Y_Test = train_test_split(X,Y, test_size=0.3, random_state=42, shuffle=True)

    if val_need:
        if Y.dtype is 'O':
            X_Val,X_Test,Y_Val,Y_Test = train_test_split(X_Test,Y_Test, test_size=0.5, random_state=42, shuffle=True, stratify=Y_test)
        else:
            X_Val,X_Test,Y_Val,Y_Test = train_test_split(X_Test,Y_Test, test_size=0.5, random_state=42, shuffle=True)
    else:
        X_Val,Y_Val = [],[]
        
    return {'X_Train':X_Train, 'X_Val':X_Val, 'X_Test':X_Test,'Y_Train':Y_Train,'Y_Test':Y_Test,'Y_Val':Y_Val}

In [266]:
Split_data = data_split(clean_data,'SalePrice',val_need=False)

In [251]:
num_cols, cat_cols = itemgetter('num_cols','cat_cols')(cleaning_results)
X_Train,Y_Train = itemgetter('X_Train','Y_Train')(Split_data)
X_Test,Y_Test = itemgetter('X_Test','Y_Test')(Split_data)

In [267]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import linear_model
from operator import itemgetter

In [241]:
# define model
model = linear_model.LinearRegression()
# define transform
transformer = ColumnTransformer(transformers=[('num',StandardScaler(),num_cols),('cat',OneHotEncoder(),cat_cols)])
# define pipeline
pipeline = Pipeline(steps=[('t', transformer), ('m',model)])
# fit the pipeline on the transformed data
pipeline.fit(train_x, train_y)

Pipeline(steps=[('t',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Id', 'MSSubClass',
                                                   'LotFrontage', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBuilt', 'YearRemodAdd',
                                                   'MasVnrArea', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'BsmtUnfSF',
                                                   'TotalBsmtSF', '1stFlrSF',
                                                   '2ndFlrSF', 'LowQualFinSF',
                                                   'GrLivArea', 'BsmtFullBath',
                                                   'BsmtHalfBath', 'FullBath',
                                                   'HalfBath', 'BedroomA...
                                             

In [268]:
# make predictions
yhat = pipeline.predict(test_x)

In [247]:
from sklearn.metrics import mean_squared_error
mean_squared_error(test_y,yhat)

3.253434117297901e+29

In [188]:
if clean_data['SalePrice'].dtype is not 'O':
    print ('Cool')

Cool


SelectKBest(score_func=<function f_regression at 0x7fca84814950>)

In [ ]:
# standard scaler - 
# minmaxscaler
# robust scaler - where there are more outliers

In [ ]:


# Applyin Data Transformations

# High accuracy and senstivity cna be assued with gaussian transformation.

use qq plot to change by logarithmic tranformation
receiprocal trasnformation
square root trasnformation
expotnetila transformation
box-cox trasnformation 

import scipy.stats as stat
import pylab

def plot_data(df,feature):
    plt.figure(figsize=(10,6))
    plot.subplot (1,2,1)
    df[feature].hist()
    plt.subplot(1,2,2)
    stat.probplot(df[feature],dist='norm',plot-pylab)
    plt.show()

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

feature_sel_model = SelectFromModel(Lasso(alpha=0.005,random_state=0))
feature_sel_model(x_Train,y_Train)
feature_sel_model.get-support()

# Always do feature engineering before train and test split

# remove categorical vlariables that are less than 1% of observations. If there are more than 2
# Try standard or minmax scaler
# save the fit features from training data and just transfomr the test data
# concant te training and test data
### ML Pipeline Steps:
#### Address the following:
##### 1. Missing Values - 
##### 2. Numerical Varaibles - Are to be scaled if they are skewed based on kurtosis factor.
##### 3. Categorical variables and their cardinality are to be studied followed by 
##### 4. Outliers
##### 5. Relation between independant and dependant feature
#
    # Idnetify the different column types
    # first find temp_cols
    # then convert all num like strings to numbers
    # use their unique value count to see if they are cat or ordinal or numeric
#     # Identify Date Time variable and adjust accordingly
    temp_cols = []

Bring in pipeline for different imputer

import numpy as np
>>> from sklearn.experimental import enable_iterative_imputer
>>> from sklearn.impute import IterativeImputer
>>> imp_mean = IterativeImputer(random_state=0)
>>> imp_mean.fit([[7, 2, 3], [4, np.nan, 6], [10, 5, 9]])
IterativeImputer(random_state=0)
>>> X = [[np.nan, 2, 3], [4, np.nan, 6], [10, np.nan, 9]]
>>> imp_mean.transform(X)
array([[ 6.9584...,  2.       ,  3.        ],
       [ 4.       ,  2.6000...,  6.        ],
       [10.       ,  4.9999...,  9.        ]])

 import numpy as np
>>> from sklearn.impute import KNNImputer
>>> X = [[1, 2, np.nan], [3, 4, 3], [np.nan, 6, 5], [8, 8, 7]]
>>> imputer = KNNImputer(n_neighbors=2)
>>> imputer.fit_transform(X)
array([[1. , 2. , 4. ],
       [3. , 4. , 3. ],
       [5.5, 6. , 5. ],
       [8. , 8. , 7. ]])

import numpy as np
>>> from sklearn.impute import SimpleImputer
>>> imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
>>> imp_mean.fit([[7, 2, 3], [4, np.nan, 6], [10, 5, 9]])
SimpleImputer()
>>> X = [[np.nan, 2, 3], [4, np.nan, 6], [10, np.nan, 9]]
>>> print(imp_mean.transform(X))
[[ 7.   2.   3. ]
 [ 4.   3.5  6. ]
 [10.   3.5  9. ]]

def Feature_select_1(X,y):
    
    from sklearn.datasets import make_regression,make_classification
    from sklearn.feature_selection import SelectKBest
    from sklearn.feature_selection import f_regression,f_classif
    
    if y.dtype is not 'o':
        # pearson's correlation feature selection for numeric input and numeric output
        X, y = make_regression(n_samples=100, n_features=100, n_informative=10)
        # define feature selection
        fs = SelectKBest(score_func=f_regression, k=10)
    else:
        # ANOVA feature selection for numeric input and categorical output
        # generate dataset
        X, y = make_classification(n_samples=100, n_features=20, n_informative=2)
        # define feature selection
        fs = SelectKBest(score_func=f_classif, k=2)
    
    X_selected = fs.fit_transform(X, y)
    return fs

Feature_select_1(Split_data['X_Train'].drop('SalePrice',axis=1), Split_data['X_Train']['SalePrice'])